In [3]:
import time
import pymongo
import pandas as pd
import requests
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
def mongo_database_setup():
    database_name={}

    # Try to connect to MongoDB,  exit if not successful.
    try:
        conn=pymongo.MongoClient()
        print "Connected successfully!!!"
        
    except pymongo.errors.ConnectionFailure, e:
       print "Could not connect to MongoDB: %s" % e 


    #Use todays date for the database name:
    name='polisci_test'+time.strftime("%d%b%y")

    if name in conn.database_names():
        conn.drop_database(name) #Drop the database if it exists
        db = conn[name] #Create the database
        
        #Create two collections in the database
        folios = db.folios
        responses=db.responses
   
    else:
        db = conn[name] #Create the database
        #Create two collections in the database
        folios = db.folios
        responses = db.responses
   
    #return the connection, database name, collections names.     
    return conn,db,folios,responses

In [4]:
conn, db, folios, responses = mongo_database_setup()

Connected successfully!!!


# How many documents do we have to scrape and download?

In [6]:
poli = conn['polisci_test26Oct16']
count = 0
poli.get_collection('folios').find({'ARCHIVORESPUESTA' : {'$regex' : 'respuesta=' }}).count()

804315

In [7]:
htmls_to_ping = []
for doc in poli.get_collection('folios').find({'ARCHIVORESPUESTA' : {'$regex' : 'respuesta=' }}).limit(150):
        htmls_to_ping.append(doc['ARCHIVORESPUESTA'])
print len(htmls_to_ping)

150


In [ ]:
countsofResponses = defaultdict(int)
for url in htmls_to_ping:
    r = requests.get(url,verify = False)
    countsofResponses[r.status_code] += 1 
    time.sleep(1) # gotta be nce to the server
print countsofResponses

### This looks suspicious. Are the last five the only 500 reposnses? Lets look at the next 50 requests and see what that gives us

In [ ]:
countsofResponses = defaultdict(int)
count = 0
for url in htmls_to_ping:
    if count >= 100:
        r = requests.get(url,verify = False)
        countsofResponses[r.status_code] += 1 
        time.sleep(1) # gotta be nce to the server
        count += 1
    else: 
        count += 1

print countsofResponses

We want to download the raw stream of bytes coming from the mex server

In [20]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True, verify = False)
    with open('./'+local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [34]:
import os
def download_file_as_zip(url,filename,path):
    infilename = download_file(url)
    # most of these files are zip files... lets just change extension to this
    newname = os.path.splitext(path+filename)[0]+'.zip'
    # yes
    output = os.rename(infilename, newname)
    print "File is located at:",newname

In [35]:
download_file_as_zip(htmls_to_ping[2],'index2','./')

File is located at: ./index2.zip
